In [544]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json 
from pandas.io.json import json_normalize

In [545]:
businesses = pd.read_csv('Business_Licenses.csv')

In [546]:
businesses.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2006,08/15/2007,08/10/2006,08/11/2006,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,...,05/16/2016,05/15/2018,05/18/2016,08/30/2016,AAI,NaN,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)"
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,...,06/16/2016,06/15/2017,06/21/2016,06/22/2016,AAC,08/30/2016,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)"
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,...,10/16/2014,10/15/2016,04/01/2016,04/01/2016,AAI,NaN,38.0,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)"
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,...,05/16/2007,05/15/2008,05/10/2007,05/11/2007,AAI,NaN,5.0,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"


In [547]:
# clean latitude and longitude columns

#dropnans 
businesses = businesses[np.isfinite(businesses['LONGITUDE'])]
businesses = businesses[np.isfinite(businesses['LATITUDE'])]

In [548]:
data = []
with open("Neighborhoods_2012_polygons.json", "r") as json_data:
    data = json.load(json_data)
df = {}
for item in data["features"]:
    if  item["type"] =='Feature':
        if 'properties' in item.keys():
            nn = item.get("properties").get("PRI_NEIGH")
            df[nn] = nn
        if 'geometry' in item:
            coords = item.get('geometry').get('coordinates')
            df[nn] = coords
df_n=pd.DataFrame(pd.Series(df))
df_n.head()
#df_n = pd.DataFrame(pd.split(""))

,0
Grand Boulevard,"[[[-87.60670812560363, 41.81681377137387], [-8..."
Printers Row,"[[[-87.62760697485339, 41.87437097785366], [-8..."
United Center,"[[[-87.66706868914592, 41.88885187769542], [-8..."
Sheffield & DePaul,"[[[-87.65833494805524, 41.921661442291786], [-..."
Humboldt Park,"[[[-87.74059567509258, 41.88782316893226], [-8..."


In [549]:
df_new = pd.DataFrame({'Neighborhood': df_n.index.values, 'Polygons': df_n[0].values})
df_new.head()

,Neighborhood,Polygons
0,Grand Boulevard,"[[[-87.60670812560363, 41.81681377137387], [-8..."
1,Printers Row,"[[[-87.62760697485339, 41.87437097785366], [-8..."
2,United Center,"[[[-87.66706868914592, 41.88885187769542], [-8..."
3,Sheffield & DePaul,"[[[-87.65833494805524, 41.921661442291786], [-..."
4,Humboldt Park,"[[[-87.74059567509258, 41.88782316893226], [-8..."


In [550]:
df_new['Polygons_tup'] = df_new['Polygons'].apply(lambda x: [tuple(a) for a in x[0]])

In [551]:
df_new.head()

,Neighborhood,Polygons,Polygons_tup
0,Grand Boulevard,"[[[-87.60670812560363, 41.81681377137387], [-8...","[(-87.60670812560363, 41.81681377137387), (-87..."
1,Printers Row,"[[[-87.62760697485339, 41.87437097785366], [-8...","[(-87.62760697485339, 41.87437097785366), (-87..."
2,United Center,"[[[-87.66706868914592, 41.88885187769542], [-8...","[(-87.66706868914592, 41.88885187769542), (-87..."
3,Sheffield & DePaul,"[[[-87.65833494805524, 41.921661442291786], [-...","[(-87.65833494805524, 41.921661442291786), (-8..."
4,Humboldt Park,"[[[-87.74059567509258, 41.88782316893226], [-8...","[(-87.74059567509258, 41.88782316893226), (-87..."


In [552]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [553]:

def neighbor_finder(lat, long):
    for index,row in df_new.iterrows():
        poly = row['Polygons_tup']
        if len(poly) >= 3:
            polygon = Polygon(poly)
            point = Point(lat, long)
            if polygon.contains(point):
                global neighborhood
                neighborhood = row['Neighborhood']
    return neighborhood

In [554]:
long = -87.723960
lat = 41.825320
neighbor_finder(long, lat)

'Little Village'

In [555]:
nhoods = []
for index, row in businesses.iterrows():
    lat = row['LATITUDE']
    long = row['LONGITUDE']
    hood = neighbor_finder(long,lat)
    #print(hood)
    nhoods.append(hood)
    
    
businesses['Neighborhood'] = nhoods    

KeyboardInterrupt: 

In [556]:
len(nhoods)

575497

In [ ]:
businesses.columns

In [ ]:
businesses['LICENSE TERM START DATE'] = pd.to_datetime(businesses['LICENSE TERM START DATE'], format='%m/%d/%Y')
businesses['LICENSE TERM EXPIRATION DATE'] = pd.to_datetime(businesses['LICENSE TERM EXPIRATION DATE'], format='%m/%d/%Y')
businesses['APPLICATION CREATED DATE'] = pd.to_datetime(businesses['APPLICATION CREATED DATE'], format='%m/%d/%Y')
businesses['APPLICATION REQUIREMENTS COMPLETE'] = pd.to_datetime(businesses['APPLICATION REQUIREMENTS COMPLETE'], format='%m/%d/%Y')
businesses['PAYMENT DATE'] = pd.to_datetime(businesses['PAYMENT DATE'], format='%m/%d/%Y')
businesses['LICENSE APPROVED FOR ISSUANCE'] = pd.to_datetime(businesses['LICENSE APPROVED FOR ISSUANCE'], format='%m/%d/%Y')
businesses['DATE ISSUED'] = pd.to_datetime(businesses['DATE ISSUED'], format='%m/%d/%Y')
businesses['LICENSE STATUS CHANGE DATE'] = pd.to_datetime(businesses['LICENSE STATUS CHANGE DATE'], format='%m/%d/%Y')

In [ ]:
businesses['BUSINESS ACTIVITY'].head()

In [ ]:
businesses['BUSINESS ACTIVITY'].value_counts()

In [ ]:
businesses['BUSINESS ACTIVITY'].isna().sum()

In [ ]:
(~businesses['BUSINESS ACTIVITY'].isna()).sum()

In [ ]:
#Get rid of all nan values in the business column
businesses = businesses.dropna()